In [37]:
import pandas as pd 
import numpy as np
import nltk 

In [2]:
data = pd.read_csv('train.csv')
data.drop('Id',axis=1 , inplace=True)
#data

In [3]:
data.shape 
data.dtypes

Comment    object
Topic      object
dtype: object

In [4]:
CommentsToBeTokenized = data['Comment'] 
CommentsToBeTokenized

0       A few things. You might have negative- frequen...
1       Is it so hard to believe that there exist part...
2                                          There are bees
3       I'm a medication technician. And that's alot o...
4                          Cesium is such a pretty metal.
                              ...                        
8690    I make similar observations over the last week...
8691                                      You would know.
8692              Also use the correct number of sig figs
8693    What about the ethical delimmas,  groundbreaki...
8694                            I would like to know too.
Name: Comment, Length: 8695, dtype: object

In [5]:
tokens = CommentsToBeTokenized[0].split()
#tokens

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
bow = cv.fit_transform(CommentsToBeTokenized)

In [7]:
bow

<8695x18177 sparse matrix of type '<class 'numpy.int64'>'
	with 190076 stored elements in Compressed Sparse Row format>

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_result=tfidf.fit_transform(CommentsToBeTokenized).toarray()

In [9]:
tfidf_result.shape

(8695, 18177)

In [45]:
vectorizer = TfidfVectorizer(max_features=6000)
tfidf_result = vectorizer.fit_transform(CommentsToBeTokenized).toarray()
features = vectorizer.get_feature_names_out()
tfidf_result = pd.DataFrame(tfidf_result, columns=features)

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
NB= MultinomialNB()

In [66]:
xtrain, xvalid, ytrain, yvalid = train_test_split(tfidf_result, data['Topic'], test_size= 0.2)
NB.fit(xtrain, ytrain)

MultinomialNB()

In [48]:
ypred_train = NB.predict(xtrain)
ypred_valid = NB.predict(xvalid)

In [49]:
ypred.shape

(1739,)

In [50]:
print("Training Results:\n")
print(classification_report(ypred_train, ytrain))
print("\nValidation Results:\n")
print(classification_report(ypred_valid, yvalid))

Training Results:

              precision    recall  f1-score   support

     Biology       0.95      0.73      0.82      3731
   Chemistry       0.75      0.84      0.80      2079
     Physics       0.62      0.95      0.75      1146

    accuracy                           0.80      6956
   macro avg       0.77      0.84      0.79      6956
weighted avg       0.84      0.80      0.80      6956


Validation Results:

              precision    recall  f1-score   support

     Biology       0.89      0.65      0.75       993
   Chemistry       0.64      0.72      0.68       522
     Physics       0.47      0.89      0.62       224

    accuracy                           0.70      1739
   macro avg       0.67      0.75      0.68      1739
weighted avg       0.76      0.70      0.71      1739



In [55]:
sparse = vectorizer.transform(["Is this question about Chemistry?"])


matrix([[0., 0., 0., ..., 0., 0., 0.]])

In [57]:
NB.predict(np.array(sparse.todense()))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(


'Chemistry'

In [63]:
def online_test(model, vectorizer, question, right_label):
    # convert the question to a vector, using an already existing vector.
    # There is a subtle issue here - if question includes a word that
    # isn't already in the dataset, the transformer/vectorizor won't handle
    # this.
    #
    # This can be solved, but with careful consideration. Read about online 
    # feature extraction for text for some solutions. 
    #
    # Online tf-idf example https://github.com/idoshlomo/online_vectorizers
    question_as_sparse_matrix = vectorizer.transform([question])
    # convert to vector, this is inefficient but OK
    question_as_vector = np.array(question_as_sparse_matrix.todense())
    
    # predict the label
    prediction = model.predict(question_as_vector)
    results = {'Comment': question, 'Topic': right_label, 'Guess' : prediction[0]}
    print(results) 
    # check if the prediction is correct
    if prediction[0] == right_label:
        return "Correct"
    else:
        return "Incorrect"

In [68]:
online_test(NB, vectorizer, "Are two atoms of the same element identical?", "Chemistry")

{'Comment': 'Are two atoms of the same element identical?', 'Topic': 'Chemistry', 'Guess': 'Chemistry'}


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(


'Correct'